# Data exploration with GEE

In [1]:
import os
import ee
import json
import ipywidgets
from IPython.display import Image
import folium

ee.Initialize()

In [2]:
def tile_url(image, viz_params=None):
    """Create a target url for tiles for an image.
    e.g.
    im = ee.Image("LE7_TOA_1YEAR/" + year).select("B3","B2","B1")
    viz = {'opacity': 1, 'gain':3.5, 'bias':4, 'gamma':1.5}
    url = tile_url(image=im),viz_params=viz)
    """
    if viz_params:
        d = image.getMapId(viz_params)
    else:
        d = image.getMapId()
    base_url = 'https://earthengine.googleapis.com'
    url = (base_url + '/map/' + d['mapid'] + '/{z}/{x}/{y}?token=' + d['token'])
    return url

### Load Features of interest

### Load GEE features

In [3]:
# MOL
mol_pa = ee.FeatureCollection("users/guizarcou/mol_pa")
mol_facets = ee.FeatureCollection("users/guizarcou/mol_patterns_facet_map")
protea = ee.Image("users/guizarcou/protea_rarity_map_v1")

# WDPA
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")

# Land cover
lc = ee.Image("users/alexguizar/ESACCI-LC-L4-LCCS-Map-300m-P1Y-1992_2015-v207")
lc_RGB = lc.visualize({'bands': ['b7', 'b5', 'b2'], 'max': 0.5})
lc_RGB

### Tile

In [5]:
# tiles_lc = tile_url(lc_RGB,{'bands': ['b7', 'b5', 'b2']}) <- not working
tiles_protea = tile_url(protea,{'opacity': 0.5, 'gain':3.5, 'bias':4, 'gamma':1.5})
# tiles_kba = tile_url(kba,{'opacity': 0.2, 'gain':3.5, 'bias':4, 'gamma':1.5})
tiles_wdpa = tile_url(wdpa,{'opacity': 0.3, 'gain':3.5, 'bias':4, 'gamma':1.5,
                           'palette': ['#00FFFF']}) # <- not making any difference

In [6]:
# Draw polygon around the Okavango Delta
OkavangoDelta = ee.Feature(
        ee.Geometry.Polygon(
            [[[21.02783203125, -20.220965779522302],
              [23.895263671875, -20.293113447544098],
              [23.994140625, -20.086888505561003],
              [23.851318359375, -18.104087015773946],
              [21.005859375, -18.145851771694467]]]),
        {
          "system:index": "0"
        })

tiles_okavango = tile_url(OkavangoDelta)

### Map

In [8]:
map = folium.Map(tiles='Mapbox Bright')
# map.add_tile_layer(tiles=tiles_lc, attr="LandCover")
map.add_tile_layer(tiles=tiles_protea, attr="Protea MoL")
# map.add_tile_layer(tiles=tiles_okavango, attr="Okavango area")
map.add_tile_layer(tiles=tiles_wdpa, attr="WDPA WCMC") 
map